In [1]:
import os
from DSSCircuit_Interface import * 
import networkx as nx

In [2]:
FolderName=os.path.dirname(os.path.realpath("__file__"))
d_obj= DSS(r""+ FolderName+ "\IEEE123Master.dss")  #creating a DSS object instance

In [3]:
d_obj.compile_ckt_dss() #compiling the circuit
d_obj.solve_snapshot_dss(1.0) #solving snapshot power flow

In [4]:
edges_dictlist=[]
G_original=nx.Graph()
for e in d_obj.get_AllPDElements():
    if ((e.split('.')[0]=='Line') or (e.split('.')[0]=='Transformer')): #only if its a line or transformer used in graph(capacitors avoided)
        branch_obj=Branch(d_obj,e) #creating  a branch object instance with full name
        sr_node=branch_obj.bus_fr.split('.')[0] #extracting source bus of branch
        tar_node=branch_obj.bus_to.split('.')[0] #extracting target bus of branch
        name=e.split('.')[1] #getting the name of the element(label or id)
        el_type=e.split('.')[0] #getting the type of the element (line or transformer)
        numphases=branch_obj.nphases #extracting number of phases of branch
        Inorm=branch_obj.Cap #extracting the baseflow current flow in branch
        Imax=branch_obj.MaxCap #extracting the maximum permissible current flow in branch
        Iresid=Imax-Inorm #residual current flow in branch
        if el_type== 'Line':
            R,X=d_obj.get_Line_RX(name)
        else:
            R,X=d_obj.get_Transformer_RX(name)
        edges_dictlist.append({"source":sr_node, "target":tar_node, "label":name, "device":el_type, "resistance":R, "reactance":X, "phases":numphases,"maxcap":Imax,"cap":Inorm,"residue":Iresid})
        G_original.add_edge(sr_node, tar_node, label=name, device=el_type, resistance=R, reactance=X, phases=numphases,maxcap=Imax,cap=Inorm,residue=Iresid)

In [6]:
buses=list(d_obj.get_AllBuses())
loads=d_obj.get_AllLoads()
# buses.remove('sourcebus') #sourcebus not in 8500

In [7]:
for b in buses:
    P_sum=0
    Q_sum=0
    for j in range(len(loads)):
        if loads[j]["Bus"]==b:
            P_sum=P_sum+loads[j].get("Pload")
            Q_sum=Q_sum+loads[j].get("Qload")
    G_original.nodes[b]["Device"]="Bus"
    G_original.nodes[b]["ActiveLoad"]=P_sum
    G_original.nodes[b]["ReactiveLoad"]=Q_sum

In [8]:
G_original.edges(data=True)

EdgeDataView([('150', '150r', {'label': 'reg1a', 'device': 'Transformer', 'resistance': 1.7305600000000006e-05, 'reactance': 0.0034611200000000007, 'phases': 3, 'maxcap': 1040.8959180101426, 'cap': 518.9463274161105, 'residue': 521.9495905940321}), ('150r', '149', {'label': 'sw1', 'device': 'Line', 'resistance': 1e-06, 'reactance': 0.0, 'phases': 3, 'maxcap': 600.0, 'cap': 518.946327388163, 'residue': 81.05367261183699}), ('149', '1', {'label': 'l115', 'device': 'Line', 'resistance': 0.023186868666666666, 'reactance': 0.047502525333333344, 'phases': 3, 'maxcap': 600.0, 'cap': 518.9463274083004, 'residue': 81.05367259169964}), ('1', '2', {'label': 'l1', 'device': 'Line', 'resistance': 0.014684974733333333, 'reactance': 0.014887152758333333, 'phases': 1, 'maxcap': 600.0, 'cap': 9.337726091174046, 'residue': 590.6622739088259}), ('1', '3', {'label': 'l2', 'device': 'Line', 'resistance': 0.020978535333333336, 'reactance': 0.021267361083333335, 'phases': 1, 'maxcap': 600.0, 'cap': 46.559664

In [10]:
nx.readwrite.gml.write_gml(G_original,"123busEx.gml")